## Test overview

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go
from vessel_manoeuvring_models.data import mdl
import shutil


In [ ]:
df_runs = pd.read_csv('../data/raw/runs.csv', index_col=0)

In [ ]:
df_runs.head()

In [ ]:
df_runs['series_number'].unique()

In [ ]:
df_runs['sailing'] = False
mask = df_runs['test_type']=='sailing'
df_runs.loc[mask,'sailing']=True
mask = df_runs['comment'].str.contains('GWA')
df_runs.loc[mask,'sailing']=True


In [ ]:
df_runs.groupby(by=['test_type','comment']).first()['sailing']

In [ ]:
df_runs.to_csv('../data/raw/runs.csv')

In [ ]:
df_runs['test_type'].unique()

In [ ]:
for test_type, group in df_runs.groupby(by=['test_type']):
    
    fig,ax=plt.subplots()
    ax.set_title(test_type)
    
    for id, row in group.iterrows():
        
        df = mdl.load_run(id=id)
        df = df.resample('3S').mean()        
        
        df.plot(x='y0', y='x0', ax=ax)
        
    ax.get_legend().set_visible(False)
    ax.axis('equal')
    ax.set_ylabel('x0')


In [ ]:
for (test_type,sailing), group in df_runs.groupby(by=['test_type','sailing']):
    
    data = pd.DataFrame()
    
    for id, row in group.iterrows():
        
        df = mdl.load_run(id=id)
        df = df.resample('2S').mean()        
        df['id'] = id    
        df['comment'] = row['comment']
                
        data = data.append(df)
    
    title = test_type
    if sailing:
        title+=' sailing'
    
    display(px.line(data, x='y0', y='x0', width=1000, height=1000, line_group='id', color='id', title=title, hover_data=['comment']))

In [ ]:
df = df_runs.groupby('sailing').get_group(False)

In [ ]:
for id, row in df.iterrows():
    
    shutil.copyfile(f'../data/raw/{id}.csv',os.path.join(r'E:\dev\wPCC_pipeline\data\01_raw',f'{id}.csv'))
    

In [ ]:
import yaml
global_dict = {}
global_dict['runs'] = list(df.index.astype(str))
with open(r'E:\dev\wPCC_pipeline\conf\base\runs_globals.yml', mode='w') as file:
    yaml.dump(global_dict, file)

In [ ]:
df.index.astype(str)